In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
from experiment_analysis import analyze_and_plot_features

In [3]:
import numpy as np

# Define fixed parameters that do not change during the analysis
fixed_params = {
    'countries': ['US', 'UK', 'DE', 'FR', 'CA', 'AU', 'JP', 'IN'],  # List of countries to be analyzed
    'platforms': ['iOS', 'Android', 'Web', 'Desktop'],  # Platforms to be included in the analysis
    'user_segments': ['Segment_1', 'Segment_2', 'Segment_3', 'Segment_4'],  # User segments to be considered
    'ab_groups': ['a1', 'a2', 'b'],  # A/B test groups
    
    'noise_level': 0.5,  # Noise level is fixed for all iterations
    'correlation_level': 0.5,  # Correlation level is fixed for all iterations
    # 'base_increase_percentage': 0.05,  # Base increase percentage is fixed for all iterations
}

# Define parameters with ranges
varying_params = {
    'base_increase_percentage': np.arange(0, 0.08, 0.01),  # Range of base increase percentages to be analyzed
}

# Define parameters with ranges to be used as the X-axis
x_params = {
    'num_users': np.arange(1000, 20000, 2000)  # Range of user numbers to be used on the X-axis
}

# Call the function to analyze and plot for all selected features
analyze_and_plot_features(
    fixed_params, 
    varying_params, 
    x_params, 
    # num_iterations=5,  # Number of iterations to perform for each analysis
    num_iterations=50,  # Original higher number of iterations, can be used for more thorough analysis
)


INFO:experiment_analysis.experiment_analysis:Analyzing feature 'num_users' with fixed params: {'countries': ['US', 'UK', 'DE', 'FR', 'CA', 'AU', 'JP', 'IN'], 'platforms': ['iOS', 'Android', 'Web', 'Desktop'], 'user_segments': ['Segment_1', 'Segment_2', 'Segment_3', 'Segment_4'], 'ab_groups': ['a1', 'a2', 'b'], 'noise_level': 0.5, 'correlation_level': 0.5, 'base_increase_percentage': 0.0}
 40%|█████████████████████████████████████████████████████▏                                                                               | 4/10 [01:23<02:05, 20.84s/it]


ValueError: Seed must be between 0 and 2**32 - 1

In [ ]:
import numpy as np

# Define fixed parameters that do not change during the analysis
fixed_params = {
    'countries': ['US', 'UK', 'DE', 'FR', 'CA', 'AU', 'JP', 'IN'],  # List of countries to be analyzed
    'platforms': ['iOS', 'Android', 'Web', 'Desktop'],  # Platforms to be included in the analysis
    'user_segments': ['Segment_1', 'Segment_2', 'Segment_3', 'Segment_4'],  # User segments to be considered
    'ab_groups': ['a1', 'a2', 'b'],  # A/B test groups
    
    'noise_level': 0.5,  # Noise level is fixed for all iterations
    # 'correlation_level': 0.5,  # Correlation level is fixed for all iterations
    'base_increase_percentage': 0.05,  # Base increase percentage is fixed for all iterations
}

# Define parameters with ranges
varying_params = {
    'correlation_level': np.arange(0, 1.1, 0.1),  # Correlation level is fixed for all iterations
}

# Define parameters with ranges to be used as the X-axis
x_params = {
    'num_users': np.arange(1000, 20000, 2000)  # Range of user numbers to be used on the X-axis
}

# Call the function to analyze and plot for all selected features
analyze_and_plot_features(
    fixed_params, 
    varying_params, 
    x_params, 
    # num_iterations=5,  # Number of iterations to perform for each analysis
    num_iterations=50,  # Original higher number of iterations, can be used for more thorough analysis
)


In [ ]:
import numpy as np

# Define fixed parameters that do not change during the analysis
fixed_params = {
    'countries': ['US', 'UK', 'DE', 'FR', 'CA', 'AU', 'JP', 'IN'],  # List of countries to be analyzed
    'platforms': ['iOS', 'Android', 'Web', 'Desktop'],  # Platforms to be included in the analysis
    'user_segments': ['Segment_1', 'Segment_2', 'Segment_3', 'Segment_4'],  # User segments to be considered
    'ab_groups': ['a1', 'a2', 'b'],  # A/B test groups
    
    # 'noise_level': 0.5,  # Noise level is fixed for all iterations
    'correlation_level': 0.5,  # Correlation level is fixed for all iterations
    'base_increase_percentage': 0.05,  # Base increase percentage is fixed for all iterations
}

# Define parameters with ranges
varying_params = {
    'noise_level': np.arange(0, 5, 0.5),  # Correlation level is fixed for all iterations
}

# Define parameters with ranges to be used as the X-axis
x_params = {
    'num_users': np.arange(1000, 20000, 2000)  # Range of user numbers to be used on the X-axis
}

# Call the function to analyze and plot for all selected features
analyze_and_plot_features(
    fixed_params, 
    varying_params, 
    x_params, 
    # num_iterations=5,  # Number of iterations to perform for each analysis
    num_iterations=50,  # Original higher number of iterations, can be used for more thorough analysis
)
